In [1]:
import pandas as pd
from tec_an import TecAn

In [2]:
tec = TecAn()

In [3]:
parsed = lambda list, price, amount: tec.add_ta(list, price, amount)

In [4]:
import numpy as np
path = "./" 

import data_util
from data_generator import DataGenerator

def load_datasets(dirs):
  sets = []  
  for raw_dir in dirs:
    data_gen = DataGenerator(random=False, first_index=10, base_dir = raw_dir)
    data_gen.rewind()
    full = data_gen.steps - 2000
    print("Loading {} from dir {}".format(full, raw_dir))
    sets.append(data_util.get_sets(data_gen, full, val_percentage = 0.003, path = path, use_cache=False, on_state_parsed = parsed))
  return sets

def conc_sets(sets):
  trainX = sets[0][0]
  trainY = sets[0][1]
  valX = sets[0][2]
  valY = sets[0][3]
  for i in range(1,  len(sets)):
    data_set = sets[i]
    trainX = np.append(data_set[0], trainX, axis = 0)
    trainY = np.append(data_set[1], trainY, axis = 0)
    valX = np.append(data_set[2], valX, axis = 0)
    valY = np.append(data_set[3], valY, axis = 0)
  return trainX, trainY, valX, valY

sets = load_datasets(["stock_data/"])
trainX, trainY, valX, valY = conc_sets(sets)
np.save(path + "train" + "X.npy", trainX)
np.save(path + "train" + "Y.npy", trainY)
np.save(path + "val" + "X.npy", valX)
np.save(path + "val" + "Y.npy", valY)



Loading 886156 from dir stock_data/


TypeError: 'range' object cannot be interpreted as an integer

In [ ]:
trainX.shape

In [ ]:
df = pd.DataFrame(trainX, columns = ['Close', 'Close2', 'Close3']) 
close = df['Close']

trainX.shape

In [ ]:
from ta.trend import *
from ta.momentum import *
from ta.volume import *
import matplotlib.pyplot as plt

plt.plot(df['Close2'], label='kst_sig')
plt.plot(df['Close3'], label='kst_sig')
plt.title('kst_sig')
plt.legend()
plt.show()

In [ ]:
plt.plot(df['Close'], label='kst_sig')
plt.title('kst_sig')
plt.legend()
plt.show()

In [ ]:
# Import math Library
import math

math.log(trainX[0][-1], 40)

In [ ]:
trainX[-1][-1]

In [ ]:
math.log(100, 200)